In [1]:
import sympy as sp

import sys,os,subprocess
sys.path.insert(0,'/work/JAM/apps/bin')
os.environ["PATH"]='/work/JAM/apps/bin:'+os.environ["PATH"]
from qgraf import QGRAF
from form import FORM,to_sympy,run_form,run_form2,to_sympy
from tools import checkdir,save,load,lprint,lpprint,evalf

import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
from sympy import mathematica_code as mcode
from sympy.utilities.lambdify import lambdastr

# Derive Cross Section

In [2]:
code = '''
Off Statistics;
Symbols M,Q,lam,E,h;
Symbols x,y;
Symbols n;
Vectors P,q,S;
Vectors Phat,Shat;
Vectors l,lp;
Autodeclare Vector V;
Tensor ghat;
Autodeclare Index o;
Autodeclare Symbols F,g;
Cfunction dot,e(antisymmetric);

L [L] = 2*(l(o1)*lp(o2) + lp(o1)*l(o2) - l.lp*d_(o1,o2) - I_*lam*e(o1,o2,o3,o4)*l(o3)*lp(o4));
L [W] = I_*e(o1,o2,o5,o6)*q(o5)*(S(o6)*g1 + (S(o6) - S.q/P.q*P(o6))*g2)/P.q
        +(((P(o1) - P.q/q^2*q(o1))*(S(o2) - S.q/q^2*q(o2)) + (S(o1) - S.q/q^2*q(o1))*(P(o2) - P.q/q^2*q(o2)))/2 
            - S.q/P.q*(P(o1) - P.q/q^2*q(o1))*(P(o2) - P.q/q^2*q(o2)))*g3/P.q
        + S.q/P.q*((P(o1) - P.q/q^2*q(o1))*(P(o2) - P.q/q^2*q(o2))*g4/P.q -(d_(o1,o2) - q(o1)*q(o2)/q^2)*g5);

L [LW] = [L]*[W];

*contract;
id e(o1,o2,o3?,o4?)*e(o1,o2,o5?,o6?) = -2*(d_(o3,o5)*d_(o4,o6) - d_(o3,o6)*d_(o4,o5));
id lp = l - q;
id P.S = 0;
id P.P = M^2;
id S.l = -h*E*M;*Q^2/2/x/y;
id S.q = -h*E*M*(y+Q^2/2/E^2);*-h*Q^2/2/x*(1+2*x^2*y*M^2/Q^2);
id E = Q^2/2/M/x/y;
id E^n? = (Q^2/2/M/x/y)^n;
id E^-n? = (Q^2/2/M/x/y)^-n;
id l.l = 0;
id q.q = -Q^2;
id P.l = P.q/y;
id P.q = Q^2/2/x;
id P.q^-1 = 2*x/Q^2;
id q.l = -Q^2/2;
id V1?.V2? = dot(V1,V2);
id V1?.V2?^n? = (dot(V1,V2))^n;
id V1?.V2?^-n? = (dot(V1,V2))^-n;

print [LW];
'''
data = run_form(code,verb=0)
print(data['LW'])
data['LW'] = data['LW'].replace(r'Q','q').replace('E','e').replace('S','s')
data['LW'] = data['LW'].replace(r'sqrt_','sqrt')
data['LW'] = to_sympy(data['LW'])

lam  = sp.symbols(r'\lambda')
h    = sp.symbols(r'h')
M    = sp.symbols(r'M')
Q    = sp.symbols(r'Q')
E    = sp.symbols(r'E')
x    = sp.symbols(r'x')
y    = sp.symbols(r'y')
alfa = sp.symbols(r'\alpha')
F1,F2,F3       = sp.symbols('F_1 F_2 F_3')
g1,g2,g3,g4,g5 = sp.symbols('g_1 g_2 g_3 g_4 g_5')


data['LW'] = data['LW'].subs(r'lam',lam)
data['LW'] = data['LW'].subs('q',Q).subs('e',E)
data['LW'] = data['LW'].subs(sp.Symbol('F1'),F1).subs(sp.Symbol('F2'),F2).subs(sp.Symbol('F3'),F3)
data['LW'] = data['LW'].subs(sp.Symbol('g1'),g1).subs(sp.Symbol('g2'),g2).subs(sp.Symbol('g3'),g3).subs(sp.Symbol('g4'),g4).subs(sp.Symbol('g5'),g5)

- 2*Q^2*h*x^-1*y^-2*g4 + 2*Q^2*h*x^-1*y^-1*g4 - 2*Q^2*h*g5 - 4*Q^2*lam*h*y^-1*g1 + 2*Q^2*lam*h*g1 - 4*M^2*h*x*y^-1*g4 + 4*M^2*h*x*y^-1*g3 + 6*M^2*h*x*g4 - 4*M^2*h*x*g3 - 4*M^2*h*x^2*y*g5 + 8*M^2*lam*h*x^2*g2 + 4*M^2*lam*h*x^2*y*g1 + 4*M^4*Q^-2*h*x^3*y*g4 - 4*M^4*Q^-2*h*x^3*y*g3


In [3]:
pol = (data['LW'].subs(h,-1) - data['LW'].subs(h,1))#/sp.S(2)

factor1  = 2*sp.pi*y*alfa**2/Q**4
factor2 = 8*sp.pi*alfa**2/x/y/Q**2

factor = factor1/factor2

M_Q = sp.Symbol(r'\left(\frac{M}{Q}\right)')

coeff4 = factor*pol.subs(g1,1).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,0).subs(M,Q*M_Q)
coeff5 = factor*pol.subs(g1,0).subs(g2,1).subs(g3,0).subs(g4,0).subs(g5,0).subs(M,Q*M_Q)
coeff6 = factor*pol.subs(g1,0).subs(g2,0).subs(g3,1).subs(g4,0).subs(g5,0).subs(M,Q*M_Q)
coeff7 = factor*pol.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,1).subs(g5,0).subs(M,Q*M_Q)
coeff8 = factor*pol.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,1).subs(M,Q*M_Q)

display(coeff4.simplify())
display(coeff5.simplify())
display(coeff6.simplify())
display(coeff7.simplify().factor(x,y))
display(coeff8.simplify().factor(x,y))

\lambda*x*y*(-y*(2*\left(\frac{M}{Q}\right)**2*x**2*y + 1) + 2)

-4*\lambda*\left(\frac{M}{Q}\right)**2*x**3*y**2

2*\left(\frac{M}{Q}\right)**2*x**2*y*(\left(\frac{M}{Q}\right)**2*x**2*y**2 + y - 1)

-(2*\left(\frac{M}{Q}\right)**2*x**2*y + 1)*(\left(\frac{M}{Q}\right)**2*x**2*y**2 + y - 1)

x*y**2*(2*\left(\frac{M}{Q}\right)**2*x**2*y + 1)

# Derive Projectors

In [60]:
code = '''
Off Statistics;
Symbols M,Q,E;
Symbols lam,h;
Symbols x,y;
Symbols n;
Vectors P,q,S;
Vectors Phat,Shat;
Autodeclare Vector V;
Autodeclare Index o;
Autodeclare Symbols g;
Cfunction dot,e(antisymmetric);

L [Wp] = I_*e(o1,o2,o3,o4)*q(o3)/P.q*(S(o4)*g1 + (S(o4) - S.q/P.q*P(o4))*g2)
            + (((P(o1)+P.q/Q^2*q(o1))*(S(o2)+S.q/Q^2*q(o2)) 
                    + (S(o1)+S.q/Q^2*q(o1))*(P(o2)+P.q/Q^2*q(o2)))/2 - S.q/P.q*(P(o1)+P.q/Q^2*q(o1))*(P(o2)+P.q/Q^2*q(o2)))*g3/P.q
            + S.q/P.q*((P(o1)+P.q/Q^2*q(o1))*(P(o2)+P.q/Q^2*q(o2))*g4/P.q - (d_(o1,o2) + q(o1)*q(o2)/Q^2)*g5);

L [Pg]   = d_(o1,o2)*[Wp];
L [Ppp]  = P(o1)*P(o2)*[Wp];
L [Pps]  = (P(o1)*S(o2)+S(o1)*P(o2))*[Wp];
L [Pepq] = I_*e(o1,o2,o5,o6)*q(o5)*P(o6)*[Wp];
L [Peqs] = I_*e(o1,o2,o5,o6)*q(o5)*S(o6)*[Wp];

*contract;
id e(o1,o2,o3?,o4?)*e(o1,o2,o5?,o6?) = -2*(d_(o3,o5)*d_(o4,o6) - d_(o3,o6)*d_(o4,o5));
id q.q = -Q^2;
id P.P = M^2;
id P.q = Q^2/2/x;
id P.q^-1 = 2*x/Q^2;
id P.S = 0;
id S.S = -M^2;
id S.q = -h*E*M*(y+Q^2/2/E^2);
id E = Q^2/2/M/x/y;
id E^n? = (Q^2/2/M/x/y)^n;
id E^-n? = (Q^2/2/M/x/y)^-n;
id h = 1;
id V1?.V2? = dot(V1,V2);
id V1?.V2?^n? = (dot(V1,V2))^(n);

print [Pg];
print [Ppp];
print [Pps];
print [Pepq];
print [Peqs];
'''
data = run_form(code,verb=0)

lam  = sp.symbols(r'\lambda')
M = sp.symbols(r'M')
Q = sp.symbols(r'Q')
x = sp.symbols(r'x')
y = sp.symbols(r'y')
g1,g2,g3,g4,g5 = sp.symbols('g_1 g_2 g_3 g_4 g_5')

for _ in data:
    data[_] = data[_].replace(r'0;','0')
    data[_] = data[_].replace(r'Q','q')
    data[_] = data[_].replace(r'sqrt_','sqrt')
    data[_] = to_sympy(data[_])
    data[_] = data[_].subs('q',Q)
    data[_] = data[_].subs('M',M)
    data[_] = data[_].subs('g1',g1).subs('g2',g2).subs('g3',g3).subs('g4',g4).subs('g5',g5)
    data[_] = data[_].simplify()#.factor(g1,g2,g3,g4,g5)
    display(data[_])

(8*M**4*x**4*y*(g_3 - g_4) + 2*M**2*Q**2*x**2*(2*g_3 - g_4*y - 2*g_4 + 6*g_5*x*y) - Q**4*g_4 + 6*Q**4*g_5*x)/(2*Q**4*x)

(32*M**6*x**6*y*(g_3 - g_4) + 8*M**4*Q**2*x**4*(g_3*y + 2*g_3 - 2*g_4*y - 2*g_4 + 2*g_5*x*y) + 4*M**2*Q**4*g_5*x**3*(y + 2) + 2*M**2*Q**4*x**2*(2*g_3 - g_4*y - 4*g_4) - Q**6*g_4 + 2*Q**6*g_5*x)/(8*Q**4*x**3)

(8*M**6*x**6*y**2*(-g_3 + 2*g_4) + 4*M**4*Q**2*x**4*(-2*g_3*y - 2*g_3 + g_4*y**2 + 4*g_4*y - 2*g_5*x*y**2) - 8*M**2*Q**4*g_5*x**3*y + 4*M**2*Q**4*x**2*(-g_3 + g_4*y + g_4) + Q**6*g_4 - 2*Q**6*g_5*x)/(4*Q**4*x**3)

(-8*M**4*g_2*x**4*y + 2*M**2*Q**2*x**2*(g_1*y - 2*g_2) + Q**4*g_1)/(Q**2*x)

(-4*M**4*g_1*x**4*y**2 + 4*M**2*Q**2*x**2*(-g_1*y + g_1 + g_2) - Q**4*g_1)/(Q**2*x)

$g_1$ projection

In [61]:
ag,app,aepq,aps,aeqs = sp.symbols(r'a^{g} a^{pp} a^{epq} a^{ps} a^{eqs}')
expr = ag*data['Pg'] + app*data['Ppp'] + aepq*data['Pepq'] + aps*data['Pps'] + aeqs*data['Peqs']

eq1 = sp.Eq(expr.subs(g1,1).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,0),1)
eq2 = sp.Eq(expr.subs(g1,0).subs(g2,1).subs(g3,0).subs(g4,0).subs(g5,0),0)
eq3 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,1).subs(g4,0).subs(g5,0),0)
eq4 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,1).subs(g5,0),0)
eq5 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,1),0)

sol = sp.solve([eq1,eq2,eq3,eq4,eq5],[ag,app,aepq,aps,aeqs])
for _ in sol.keys():
    sol[_] = sol[_].simplify().factor()
    display(sp.Eq(_,sol[_]))

Eq(a^{epq}, -Q**4/(4*M**2*x*(2*M**2*x**2*y + Q**2)*(M**2*x**2*y**2 + Q**2*y - Q**2)))

Eq(a^{eqs}, -Q**2/(4*M**2*x*(M**2*x**2*y**2 + Q**2*y - Q**2)))

Eq(a^{g}, 0)

Eq(a^{pp}, 0)

Eq(a^{ps}, 0)

$g_2$ projection

In [62]:
ag,app,aepq,aps,aeqs = sp.symbols(r'a^{g} a^{pp} a^{epq} a^{ps} a^{eqs}')
expr = ag*data['Pg'] + app*data['Ppp'] + aepq*data['Pepq'] + aps*data['Pps'] + aeqs*data['Peqs']

eq1 = sp.Eq(expr.subs(g1,1).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,0),0)
eq2 = sp.Eq(expr.subs(g1,0).subs(g2,1).subs(g3,0).subs(g4,0).subs(g5,0),1)
eq3 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,1).subs(g4,0).subs(g5,0),0)
eq4 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,1).subs(g5,0),0)
eq5 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,1),0)

sol = sp.solve([eq1,eq2,eq3,eq4,eq5],[ag,app,aepq,aps,aeqs])
for _ in sol.keys():
    sol[_] = sol[_].simplify().factor()
    display(sp.Eq(_,sol[_]))

Eq(a^{epq}, -Q**2*(2*M**2*x**2*y - 2*M*Q*x + Q**2)*(2*M**2*x**2*y + 2*M*Q*x + Q**2)/(16*M**4*x**3*(2*M**2*x**2*y + Q**2)*(M**2*x**2*y**2 + Q**2*y - Q**2)))

Eq(a^{eqs}, -Q**4/(16*M**4*x**3*(M**2*x**2*y**2 + Q**2*y - Q**2)))

Eq(a^{g}, 0)

Eq(a^{pp}, 0)

Eq(a^{ps}, 0)

$g_3$ projection

In [63]:
ag,app,aepq,aps,aeqs = sp.symbols(r'a^{g} a^{pp} a^{epq} a^{ps} a^{eqs}')
expr = ag*data['Pg'] + app*data['Ppp'] + aepq*data['Pepq'] + aps*data['Pps'] + aeqs*data['Peqs']

eq1 = sp.Eq(expr.subs(g1,1).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,0),0)
eq2 = sp.Eq(expr.subs(g1,0).subs(g2,1).subs(g3,0).subs(g4,0).subs(g5,0),0)
eq3 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,1).subs(g4,0).subs(g5,0),1)
eq4 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,1).subs(g5,0),0)
eq5 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,1),0)

sol = sp.solve([eq1,eq2,eq3,eq4,eq5],[ag,app,aepq,aps,aeqs])
for _ in sol.keys():
    sol[_] = sol[_].simplify().factor()
    display(sp.Eq(_,sol[_]))

Eq(a^{epq}, 0)

Eq(a^{eqs}, 0)

Eq(a^{g}, 0)

Eq(a^{pp}, Q**4*(2*M**2*x**2*y + Q**2)/(M**4*x*(4*M**2*x**2 + Q**2)*(M**2*x**2*y**2 + Q**2*y - Q**2)))

Eq(a^{ps}, Q**4/(2*M**4*x*(M**2*x**2*y**2 + Q**2*y - Q**2)))

$g_4$ projection

In [67]:
ag,app,aepq,aps,aeqs = sp.symbols(r'a^{g} a^{pp} a^{epq} a^{ps} a^{eqs}')
expr = ag*data['Pg'] + app*data['Ppp'] + aepq*data['Pepq'] + aps*data['Pps'] + aeqs*data['Peqs']

eq1 = sp.Eq(expr.subs(g1,1).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,0),0)
eq2 = sp.Eq(expr.subs(g1,0).subs(g2,1).subs(g3,0).subs(g4,0).subs(g5,0),0)
eq3 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,1).subs(g4,0).subs(g5,0),0)
eq4 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,1).subs(g5,0),1)
eq5 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,1),0)

sol = sp.solve([eq1,eq2,eq3,eq4,eq5],[ag,app,aepq,aps,aeqs])
for _ in sol.keys():
    sol[_] = sol[_].simplify().factor()
    display(sp.Eq(_,sol[_]))

Eq(a^{epq}, 0)

Eq(a^{eqs}, 0)

Eq(a^{g}, Q**4*x/((4*M**2*x**2 + Q**2)*(2*M**2*x**2*y + Q**2)))

Eq(a^{pp}, 4*Q**4*x*(M**4*x**4*y**2 + M**2*Q**2*x**2*y + 3*M**2*Q**2*x**2 + Q**4)/(M**2*(4*M**2*x**2 + Q**2)**2*(2*M**2*x**2*y + Q**2)*(M**2*x**2*y**2 + Q**2*y - Q**2)))

Eq(a^{ps}, 2*Q**4*x/(M**2*(4*M**2*x**2 + Q**2)*(M**2*x**2*y**2 + Q**2*y - Q**2)))

$g_5$ projection

In [65]:
ag,app,aepq,aps,aeqs = sp.symbols(r'a^{g} a^{pp} a^{epq} a^{ps} a^{eqs}')
expr = ag*data['Pg'] + app*data['Ppp'] + aepq*data['Pepq'] + aps*data['Pps'] + aeqs*data['Peqs']

eq1 = sp.Eq(expr.subs(g1,1).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,0),0)
eq2 = sp.Eq(expr.subs(g1,0).subs(g2,1).subs(g3,0).subs(g4,0).subs(g5,0),0)
eq3 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,1).subs(g4,0).subs(g5,0),0)
eq4 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,1).subs(g5,0),0)
eq5 = sp.Eq(expr.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,1),1)

sol = sp.solve([eq1,eq2,eq3,eq4,eq5],[ag,app,aepq,aps,aeqs])
for _ in sol.keys():
    sol[_] = sol[_].simplify().factor()
    display(sp.Eq(_,sol[_]))

Eq(a^{epq}, 0)

Eq(a^{eqs}, 0)

Eq(a^{g}, Q**2/(2*(2*M**2*x**2*y + Q**2)))

Eq(a^{pp}, -2*Q**2*x**2/((4*M**2*x**2 + Q**2)*(2*M**2*x**2*y + Q**2)))

Eq(a^{ps}, 0)

# Check Projectors

In [73]:
code = '''
Off Statistics;
Autodeclare Symbols a,v;
Autodeclare Index o;
Vector S,q,P;
Vector s,p,p1;
Symbol x,y,h,E,M,Q;
Symbol n;
Autodeclare Symbol g;
Autodeclare Vector V;
CFunction dot(symmetric),e(antisymmetric);

L [Wp] = I_*e(o1,o2,o3,o4)*q(o3)/P.q*(S(o4)*g1 + (S(o4) - S.q/P.q*P(o4))*g2)
            + (((P(o1)+P.q/Q^2*q(o1))*(S(o2)+S.q/Q^2*q(o2)) 
                    + (S(o1)+S.q/Q^2*q(o1))*(P(o2)+P.q/Q^2*q(o2)))/2 - S.q/P.q*(P(o1)+P.q/Q^2*q(o1))*(P(o2)+P.q/Q^2*q(o2)))*g3/P.q
            + S.q/P.q*((P(o1)+P.q/Q^2*q(o1))*(P(o2)+P.q/Q^2*q(o2))*g4/P.q - (d_(o1,o2) + q(o1)*q(o2)/Q^2)*g5);

L [Pg]   = d_(o1,o2)*[Wp];
L [Ppp]  = P(o1)*P(o2)*[Wp];
L [Pps]  = (P(o1)*S(o2)+S(o1)*P(o2))*[Wp];
L [Pepq] = I_*e(o1,o2,o5,o6)*q(o5)*P(o6)*[Wp];
L [Peqs] = I_*e(o1,o2,o5,o6)*q(o5)*S(o6)*[Wp];

L [P1] = -Q^4/(4*M^2*x)/(2*M^2*x^2*y+Q^2)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Pepq]
         - Q^2/(4*M^2*x)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Peqs];
L [P2] = -Q^2*(2*M^2*x^2*y-2*M*Q*x+Q^2)*(2*M^2*x^2*y+2*M*Q*x+Q^2)/(16*M^4*x^3)/(2*M^2*x^2*y+Q^2)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Pepq]
         - Q^4/(16*M^4*x^3)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Peqs];
L [P3] = Q^4*(2*M^2*x^2*y+Q^2)/(M^4*x)/(4*M^2*x^2+Q^2)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Ppp]
         + Q^4/(2*M^4*x)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Pps];
L [P4] = Q^4*x/(4*M^2*x^2+Q^2)/(2*M^2*x^2*y+Q^2)*[Pg]
         + 4*Q^4*x*(M^4*x^4*y^2+M^2*Q^2*x^2*y+3*M^2*Q^2*x^2+Q^4)/M^2/(4*M^2*x^2+Q^2)^2/(2*M^2*x^2*y+Q^2)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Ppp]
         + 2*Q^4*x/M^2/(4*M^2*x^2+Q^2)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Pps];
L [P5] = Q^2/2/(2*M^2*x^2*y+Q^2)*[Pg]
         - 2*Q^2*x^2/(4*M^2*x^2+Q^2)/(2*M^2*x^2*y+Q^2)*[Ppp];

*contract;
id e(o1,o2,o3?,o4?)*e(o1,o2,o5?,o6?) = -2*(d_(o3,o5)*d_(o4,o6) - d_(o3,o6)*d_(o4,o5));
id p = x*P;
id s = x*S;
id e(q,P,s,p) = e(q,P,s,P)*x;
id e(S,q,s,p) = e(S,q,S,P)*x^2;
id q.q = -Q^2;
id P.P = M^2;
id P.q = Q^2/2/x;
id P.q^-1 = 2*x/Q^2;
id P.S = 0;
id S.S = -M^2;
id S.q = -h*E*M*(y+Q^2/2/E^2);
id E = Q^2/2/M/x/y;
id E^n? = (Q^2/2/M/x/y)^n;
id E^-n? = (Q^2/2/M/x/y)^-n;
id h = 1;
id V1?.V2? = dot(V1,V2);
id V1?.V2?^n? = (dot(V1,V2))^(n);

Print [P1];
Print [P2];
Print [P3];
Print [P4];
Print [P5];
'''
data = run_form(code,verb=0)

M = sp.symbols(r'M')
h = sp.symbols(r'h')
Q = sp.symbols(r'Q')
x = sp.symbols(r'x')
y = sp.symbols(r'y')
xi = sp.symbols(r'\xi')
g1,g2,g3,g4,g5 = sp.symbols('g_(1:6)')

for _ in data:
    data[_] = data[_].replace('0;','0')
    data[_] = data[_].replace(r'Q','q')
    data[_] = data[_].replace(r'sqrt_','sqrt')
    data[_] = to_sympy(data[_])
    data[_] = data[_].subs('q',Q)
    display(data[_].simplify())

g1

g2

g3

g4

g5

# Calculate Hadronic Tensor Directly

In [88]:
code = '''
Off Statistics;
Autodeclare Symbols a,v;
Autodeclare Index o;
Vector S,q,P;
Vector s,p,p1;
Symbol x,y,h,E,M,Q;
Symbol n;
Autodeclare Vector V;
CFunction dot(symmetric),e(antisymmetric);

L [What] = -2*a1*v2*(2*s(o1)*p(o2)-d_(o1,o2)*s.q) - 2*a2*v1*(2*p(o1)*s(o2)-d_(o1,o2)*s.q)
           -2*(a1*v2+a2*v1)*(s(o1)*q(o2)+q(o1)*s(o2))
           - 2*I_*e(o1,o2,o3,o4)*(2*a1*a2*s(o3)*p(o4) + (a1*a2+v1*v2)*s(o3)*q(o4));

L [Pg]   = d_(o1,o2)*[What];
L [Ppp]  = P(o1)*P(o2)*[What];
L [Pps]  = (P(o1)*S(o2)+S(o1)*P(o2))*[What];
L [Pepq] = I_*e(o1,o2,o5,o6)*q(o5)*P(o6)*[What];
L [Peqs] = I_*e(o1,o2,o5,o6)*q(o5)*S(o6)*[What];

L [P1] = -Q^4/(4*M^2*x)/(2*M^2*x^2*y+Q^2)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Pepq]
         - Q^2/(4*M^2*x)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Peqs];
L [P2] = -Q^2*(2*M^2*x^2*y-2*M*Q*x+Q^2)*(2*M^2*x^2*y+2*M*Q*x+Q^2)/(16*M^4*x^3)/(2*M^2*x^2*y+Q^2)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Pepq]
         - Q^4/(16*M^4*x^3)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Peqs];
L [P3] = Q^4*(2*M^2*x^2*y+Q^2)/(M^4*x)/(4*M^2*x^2+Q^2)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Ppp]
         + Q^4/(2*M^4*x)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Pps];
L [P4] = Q^4*x/(4*M^2*x^2+Q^2)/(2*M^2*x^2*y+Q^2)*[Pg]
         + 4*Q^4*x*(M^4*x^4*y^2+M^2*Q^2*x^2*y+3*M^2*Q^2*x^2+Q^4)/M^2/(4*M^2*x^2+Q^2)^2/(2*M^2*x^2*y+Q^2)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Ppp]
         + 2*Q^4*x/M^2/(4*M^2*x^2+Q^2)/(M^2*x^2*y^2+Q^2*y-Q^2)*[Pps];
L [P5] = Q^2/2/(2*M^2*x^2*y+Q^2)*[Pg]
         - 2*Q^2*x^2/(4*M^2*x^2+Q^2)/(2*M^2*x^2*y+Q^2)*[Ppp];

*contract;
id e(o1,o2,o3?,o4?)*e(o1,o2,o5?,o6?) = -2*(d_(o3,o5)*d_(o4,o6) - d_(o3,o6)*d_(o4,o5));
id p = x*P;
id s = x*S;
id e(q,P,s,p) = e(q,P,s,P)*x;
id e(S,q,s,p) = e(S,q,S,P)*x^2;
id q.q = -Q^2;
id P.P = M^2;
id P.q = Q^2/2/x;
id P.q^-1 = 2*x/Q^2;
id P.S = 0;
id S.S = -M^2;
id S.q = -h*E*M*(y+Q^2/2/E^2);
id E = Q^2/2/M/x/y;
id E^n? = (Q^2/2/M/x/y)^n;
id E^-n? = (Q^2/2/M/x/y)^-n;
id h = 1;
id V1?.V2? = dot(V1,V2);
id V1?.V2?^n? = (dot(V1,V2))^(n);

Print [P1];
Print [P2];
Print [P3];
Print [P4];
Print [P5];
'''
data = run_form(code,verb=0)

M = sp.symbols(r'M')
h = sp.symbols(r'h')
Q = sp.symbols(r'Q')
x = sp.symbols(r'x')
y = sp.symbols(r'y')
xi = sp.symbols(r'\xi')
g1,g2,g3,g4,g5 = sp.symbols('g_(1:6)')#' g_2 g_3 g_4 g_5')
a1,a2 = sp.symbols(r'a_(1:3)')
v1,v2 = sp.symbols(r'v_(1:3)')

for _ in data:
    data[_] = data[_].replace('0;','0')
#     print(data[_])
    data[_] = data[_].replace(r'Q','q')
    data[_] = data[_].replace(r'sqrt_','sqrt')
    data[_] = data[_].replace(r'i','I')
    data[_] = to_sympy(data[_])
    data[_] = data[_].subs(r'a1',a1).subs(r'a2',a2).subs(r'v1',v1).subs(r'v2',v2)
    data[_] = data[_].subs('q',Q)
    data[_] = data[_].subs('h',1)
    data[_] = data[_].subs('M',M)
    data[_] = data[_]*2*sp.pi/Q**2 # from on-shell delta function
    data[_] = data[_]/sp.S(4)/sp.pi # from hadron tensor normalization
    data[_] = data[_].simplify().factor().series(M,0,1).removeO().simplify().factor()
    display(data[_])

(a_1*a_2 + v_1*v_2)/2

-a_1*a_2/2

-x*(2*y - 3)*(a_1*v_2 + a_2*v_1)/(y - 1)

-x*(a_1*v_2 + a_2*v_1)

-(a_1*v_2 + a_2*v_1)/2

In [89]:
gV   = sp.symbols(r'g_{V}^{i}')
gA   = sp.symbols(r'g_{A}^{i}')
f    = sp.Symbol(r'\Delta f_i',commutative=False)
sig  = sp.Symbol(r'\Sigma_{i}',commutative=False)
eq   = sp.Symbol(r'e_{q}',commutative=False)

$\gamma$

In [97]:
display(sp.Eq(g1,sig*eq**2*data['P1'].subs(a1,0).subs(a2,0).subs(v1,1).subs(v2,1)*f))
# display(sp.Eq(g2,sig*eq**2*data['P2'].subs(a1,0).subs(a2,0).subs(v1,1).subs(v2,1)*f))
# display(sp.Eq(g3,sig*eq**2*data['P3'].subs(a1,0).subs(a2,0).subs(v1,1).subs(v2,1)*f))
# display(sp.Eq(g4,sig*eq**2*data['P4'].subs(a1,0).subs(a2,0).subs(v1,1).subs(v2,1)*f))
display(sp.Eq(g5,sig*eq**2*data['P5'].subs(a1,0).subs(a2,0).subs(v1,1).subs(v2,1)*f))

Eq(g_1, \Sigma_{i}*e_{q}**2*\Delta f_i/2)

Eq(g_5, 0)

$\gamma Z$

In [98]:
display(sp.Eq(g1,sig*eq*data['P1'].subs(a1,0).subs(a2,2*gA).subs(v1,1).subs(v2,2*gV)*f))
# display(sp.Eq(g2,sig*eq*data['P2'].subs(a1,0).subs(a2,2*gA).subs(v1,1).subs(v2,2*gV)*f))
# display(sp.Eq(g3,sig*eq*data['P3'].subs(a1,0).subs(a2,2*gA).subs(v1,1).subs(v2,2*gV)*f))
# display(sp.Eq(g4,sig*eq*data['P4'].subs(a1,0).subs(a2,2*gA).subs(v1,1).subs(v2,2*gV)*f))
display(sp.Eq(g5,sig*eq*data['P5'].subs(a1,0).subs(a2,2*gA).subs(v1,1).subs(v2,2*gV)*f))

Eq(g_1, g_{V}^{i}*\Sigma_{i}*e_{q}*\Delta f_i)

Eq(g_5, -g_{A}^{i}*\Sigma_{i}*e_{q}*\Delta f_i)

$Z$

In [99]:
display(sp.Eq(g1,(sig*data['P1'].subs(a1,gA).subs(a2,gA).subs(v1,gV).subs(v2,gV)*f).simplify()))
# display(sp.Eq(g2,(sig*data['P2'].subs(a1,gA).subs(a2,gA).subs(v1,gV).subs(v2,gV)*f).simplify()))
# display(sp.Eq(g3,(sig*data['P3'].subs(a1,gA).subs(a2,gA).subs(v1,gV).subs(v2,gV)*f).simplify()))
# display(sp.Eq(g4,(sig*data['P4'].subs(a1,gA).subs(a2,gA).subs(v1,gV).subs(v2,gV)*f)).simplify())
display(sp.Eq(g5,(sig*data['P5'].subs(a1,gA).subs(a2,gA).subs(v1,gV).subs(v2,gV)*f)).simplify())

Eq(g_1, (g_{A}^{i}**2 + g_{V}^{i}**2)*\Sigma_{i}*\Delta f_i/2)

Eq(g_5, -g_{A}^{i}*g_{V}^{i}*\Sigma_{i}*\Delta f_i)